# Notebook 7: TEORIA - Vectorización Completa de Monte Carlo

## Objetivo
Este notebook implementa los conceptos teóricos de vectorización completa del notebook de ejemplo (`MonteCarlo_Monos_enunciado.ipynb`), aplicándolos al contexto del Notebook 6 para maximizar la velocidad de la simulación de Monte Carlo.

## Conceptos Teóricos Implementados
1. **Multiplicación de matrices para cálculo de retornos**: Usar `(días × activos) @ (activos × monos) = (días × monos)` para calcular TODOS los retornos de TODOS los monos en una sola operación
2. **Selección aleatoria vectorizada con argsort**: Generar matriz aleatoria `(n_monos × n_elegibles)` y usar `np.argsort` para obtener top N índices por fila
3. **Construcción de matriz de pesos vectorizada**: Matriz de pesos `(n_monos × n_assets)` asignada usando advanced indexing vectorizado
4. **Cálculo acumulado vectorizado**: Usar `np.sum` sobre todos los monos simultáneamente

## Estrategia Híbrida
- Pre-calcular TODAS las selecciones aleatorias para TODOS los meses de una vez (vectorizado)
- Construir matriz de pesos y usar multiplicación de matrices para calcular retornos
- Procesar mes a mes pero vectorizando completamente cada mes
- Sin loops sobre monos, solo sobre meses si es necesario

## Contenido
1. Configuración y carga de datos
2. Pre-cálculo de índices elegibles
3. Monte Carlo con vectorización completa (multiplicación de matrices)
4. Análisis de resultados
5. Comparación de velocidad con Notebook 6

---

## 1. Configuración y Carga de Datos {#configuracion}

Esta sección configura el entorno y carga los datos necesarios para la simulación de Monte Carlo con vectorización completa.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy import stats

# Configuración
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Parámetros
CAPITAL_INICIAL = 250_000
DATA_BACKTEST_DIR = '../datos/backtest'
DATA_PROCESSED_DIR = '../datos/processed'

print("="*70)
print("NOTEBOOK 7: TEORIA - VECTORIZACIÓN COMPLETA")
print("="*70)

# Cargar resultados del backtesting (para comparación)
df_equity = pd.read_parquet(f'{DATA_BACKTEST_DIR}/equity_curve.parquet')
df_trades = pd.read_csv(f'{DATA_BACKTEST_DIR}/trades.csv', parse_dates=['fecha'])

# Convertir fecha a índice para análisis temporal
df_equity['fecha'] = pd.to_datetime(df_equity['fecha'])
df_equity = df_equity.set_index('fecha').sort_index()

# Calcular retornos mensuales
df_equity['retorno_mensual'] = df_equity['equity'].pct_change()

print(f"\nEquity curve: {df_equity.shape}")
print(f"Trades: {df_trades.shape}")
print(f"Período: {df_equity.index[0].date()} a {df_equity.index[-1].date()}")
print(f"Capital inicial: ${CAPITAL_INICIAL:,.0f}")
print(f"Capital final: ${df_equity['equity'].iloc[-1]:,.0f}")
print(f"Retorno total: {(df_equity['equity'].iloc[-1] / CAPITAL_INICIAL - 1) * 100:.2f}%")

NOTEBOOK 7: TEORIA - VECTORIZACIÓN COMPLETA


FileNotFoundError: [Errno 2] No such file or directory: '../datos/backtest/equity_curve.parquet'

## 2. Carga de Datos para Monte Carlo {#carga-datos}

Carga de datos necesarios para la simulación de Monte Carlo con eligibility mask.

In [8]:
# Cargar datos para MC
print("\n" + "="*70)
print("CARGANDO DATOS PARA MONTE CARLO")
print("="*70)

# Cargar retornos mensuales (WIDE format)
log_returns_monthly = pd.read_parquet(f'{DATA_PROCESSED_DIR}/log_returns_monthly.parquet')

# Cargar eligibility mask
eligibility_mask = pd.read_parquet(f'{DATA_PROCESSED_DIR}/eligibility_mask.parquet')

# Cargar fechas de rebalanceo
rebalance_dates = pd.read_csv(f'{DATA_PROCESSED_DIR}/rebalance_dates.csv', 
                               parse_dates=['date'])['date']

# Alinear retornos con fechas de rebalanceo
returns_mc = log_returns_monthly.reindex(rebalance_dates)
eligibility_mc = eligibility_mask.reindex(rebalance_dates)

print(f"Retornos para MC: {returns_mc.shape}")
print(f"Eligibility mask: {eligibility_mc.shape}")
print(f"Fechas rebalanceo: {len(rebalance_dates)}")
print(f"\n✓ Datos cargados correctamente")


CARGANDO DATOS PARA MONTE CARLO
Retornos para MC: (133, 845)
Eligibility mask: (133, 845)
Fechas rebalanceo: 133

✓ Datos cargados correctamente


## 3. Pre-cálculo de Índices Elegibles {#pre-calculo}

Pre-cálculo de índices elegibles por mes para optimizar el acceso durante la simulación.

In [9]:
# Convertir a arrays numpy
retornos_array = returns_mc.values  # (n_meses, n_tickers)
elegibles_array = eligibility_mc.values  # (n_meses, n_tickers) boolean

n_meses = retornos_array.shape[0]
n_tickers = retornos_array.shape[1]

# Pre-calcular índices elegibles por mes
print("\n" + "="*70)
print("PRE-CALCULANDO ÍNDICES ELEGIBLES")
print("="*70)
indices_elegibles_por_mes = []
n_elegibles_por_mes = []
for mes_idx in range(n_meses):
    elegibles_mes = elegibles_array[mes_idx]
    indices_elegibles = np.where(elegibles_mes)[0]
    indices_elegibles_por_mes.append(indices_elegibles)
    n_elegibles_por_mes.append(len(indices_elegibles))
print(f"✓ Índices pre-calculados para {n_meses} meses")
print(f"  Rango de elegibles por mes: {min(n_elegibles_por_mes)} - {max(n_elegibles_por_mes)}")


PRE-CALCULANDO ÍNDICES ELEGIBLES
✓ Índices pre-calculados para 133 meses
  Rango de elegibles por mes: 497 - 506


## 4. Monte Carlo con Vectorización Completa Optimizada {#monte-carlo}

Esta sección implementa el test de Monte Carlo con **optimizaciones críticas** que eliminan redundancias y maximizan la velocidad.

### Optimizaciones Aplicadas:

1. **Extracción directa de retornos con índices**: 
   - Usa `retornos_array[mes_idx, indices_seleccionados]` para extraer solo los retornos necesarios
   - Shape: `(n_monos × 20)` - solo los retornos que se necesitan, sin ceros
   - Evita construir matriz completa de pesos `(n_monos × n_assets)` con mayoría de ceros

2. **Cálculo de promedio equiponderado con `np.mean()`**:
   - `np.mean(retornos_seleccionados, axis=1)` es matemáticamente equivalente a multiplicación de matrices con pesos equiponderados
   - Porque: `mean = sum/20 = sum × 0.05` (equivalente a pesos de 5% cada uno)
   - Más eficiente que: `(1 × 845) @ (845 × 50,000)` con mayoría de ceros

3. **Selección aleatoria vectorizada mes a mes**:
   - Genera selecciones cuando se necesitan (no pre-calcula para todos los meses)
   - Usa `np.argsort` sobre matriz aleatoria `(n_monos × n_elegibles)`
   - Sin loops sobre monos

4. **Cálculo acumulado vectorizado**:
   - Usa `np.sum` sobre todos los monos simultáneamente
   - Sin loops sobre monos

### Estrategia de Implementación:

- **Generar selecciones mes a mes** cuando se necesiten (no pre-calcular)
- **Extraer retornos directamente** usando advanced indexing: solo `(n_monos × 20)` elementos
- **Calcular promedio equiponderado** con `np.mean()` en lugar de multiplicación de matrices
- **Sin loops sobre monos**, solo loop sobre meses (necesario por eligibility_mask variable)

### Ventajas de esta Optimización:

- **Memoria**: Reduce de ~42M elementos (50,000 × 845) a ~1M elementos (50,000 × 20) por mes
- **Velocidad**: Elimina construcción de matrices grandes y multiplicaciones ineficientes
- **Eficiencia**: Trabaja solo con los datos necesarios (2.4% útil vs 100% de ceros)

In [10]:
"""
Monte Carlo con Vectorización Completa Optimizada

Cada mono:
1. Vende TODA su posición (100%) cada mes (full rebalance)
2. Compra 20 activos ALEATORIOS de los ELEGIBLES ese mes
3. Usa mismo universo que la estrategia (fair comparison)
4. Paga 0.46% por rebalanceo (0.23% venta + 0.23% compra)

OPTIMIZACIONES CRÍTICAS APLICADAS:
- Extracción directa de retornos con índices (solo n_monos × 20 elementos)
- Cálculo de promedio equiponderado con np.mean() (equivalente a multiplicación de matrices)
- Selección aleatoria vectorizada mes a mes (sin pre-cálculo innecesario)
- Sin construcción de matriz completa de pesos (evita 42M elementos con mayoría de ceros)
- Cálculo acumulado vectorizado (sin loops sobre monos)
"""

# Configuración
N_SIMULACIONES = 25_000_000
BATCH_SIZE = 50_000  # Batch grande para máxima vectorización
N_ACTIVOS_SELECCION = 20
PESO_POR_ACTIVO = 1.0 / N_ACTIVOS_SELECCION  # 5% = 0.05 (equiponderado)
COSTE_REBALANCEO = 0.0046  # 0.46% = 0.23% venta + 0.23% compra (full rebalance cada mes)
PROGRESS_INTERVAL = 25_000  # Avisar cada 0.1% (25,000 de 25M)

print("="*70)
print("CONFIGURACIÓN MONTE CARLO (VECTORIZACIÓN COMPLETA)")
print("="*70)
print(f"Simulaciones: {N_SIMULACIONES:,}")
print(f"Batch size: {BATCH_SIZE:,}")
print(f"Activos por cartera: {N_ACTIVOS_SELECCION}")
print(f"Peso por activo: {PESO_POR_ACTIVO*100:.1f}% (equiponderado)")
print(f"Rebalanceo: FULL REBALANCE cada mes (vender todo, comprar todo)")
print(f"Coste rebalanceo: {COSTE_REBALANCEO*100:.2f}% (0.23% venta + 0.23% compra)")
print(f"Requisito: < 24 horas")

CONFIGURACIÓN MONTE CARLO (VECTORIZACIÓN COMPLETA)
Simulaciones: 25,000,000
Batch size: 50,000
Activos por cartera: 20
Peso por activo: 5.0% (equiponderado)
Rebalanceo: FULL REBALANCE cada mes (vender todo, comprar todo)
Coste rebalanceo: 0.46% (0.23% venta + 0.23% compra)
Requisito: < 24 horas


In [11]:
print("\n" + "="*70)
print("INICIANDO MONTE CARLO CON VECTORIZACIÓN COMPLETA OPTIMIZADA")
print(f"Simulaciones: {N_SIMULACIONES:,}")
print(f"Batch size: {BATCH_SIZE:,}")
print("="*70)

# Verificar que los datos estén cargados
try:
    returns_mc
    eligibility_mc
except NameError:
    raise NameError("ERROR: Ejecuta primero las celdas anteriores que cargan los datos")

# Pre-calcular constantes
años = n_meses / 12.0
factor_cagr = 1 / años

# Array de resultados
resultados_cagr = np.zeros(N_SIMULACIONES, dtype=np.float32)

# Crear generador de números aleatorios optimizado
rng = np.random.default_rng()

# Calcular número de batches
n_batches = N_SIMULACIONES // BATCH_SIZE

print(f"\nIniciando procesamiento de {n_batches} batches (batch_size={BATCH_SIZE:,})...")
print(f"Progress cada 0.1% ({PROGRESS_INTERVAL:,} simulaciones)\n")
print("OPTIMIZACIONES APLICADAS:")
print("  - Extracción directa de retornos (solo n_monos × 20 elementos)")
print("  - np.mean() en lugar de multiplicación de matrices (equivalente matemático)")
print("  - Sin construcción de matriz completa de pesos (evita 42M elementos con ceros)")
print("  - Selecciones generadas mes a mes (sin pre-cálculo innecesario)\n")

# Iniciar timer
start_time = time.time()

# ========== LOOP SOBRE BATCHES (OPTIMIZADO) ==========
for batch_idx in range(n_batches):
    
    n_monos_batch = BATCH_SIZE if batch_idx < n_batches - 1 else N_SIMULACIONES - (batch_idx * BATCH_SIZE)
    
    # Retornos acumulados para este batch: (n_monos,)
    retornos_acumulados = np.zeros(n_monos_batch, dtype=np.float32)
    
    # ========== PROCESAR MES A MES (sin pre-calcular selecciones) ==========
    for mes_idx in range(n_meses):
        indices_elegibles = indices_elegibles_por_mes[mes_idx]
        n_elegibles = n_elegibles_por_mes[mes_idx]
        
        if n_elegibles < N_ACTIVOS_SELECCION:
            # Caso borde: menos elegibles que necesarios
            if batch_idx == 0 and mes_idx == 0:
                print(f"ADVERTENCIA: Mes {mes_idx}, solo {n_elegibles} elegibles")
            if n_elegibles == 0:
                # No hay elegibles, retorno = 0 para este mes
                continue
            # Repetir los elegibles disponibles
            indices_seleccionados = np.tile(indices_elegibles, (n_monos_batch, 1))[:, :N_ACTIVOS_SELECCION]
        else:
            # ========== OPTIMIZACIÓN 1: Generar selecciones aleatorias (vectorizado) ==========
            # Generar matriz aleatoria: (n_monos × n_elegibles)
            rand_matrix = rng.random((n_monos_batch, n_elegibles))
            
            # Usar argsort para obtener top 20 índices por fila (vectorizado)
            indices_permutados = np.argsort(rand_matrix, axis=1)
            
            # Seleccionar top 20: (n_monos × 20)
            indices_seleccionados = indices_elegibles[indices_permutados[:, :N_ACTIVOS_SELECCION]]
        
        # ========== OPTIMIZACIÓN 2: Extraer retornos directamente usando índices ==========
        # Extraer retornos de los 20 activos seleccionados para cada mono
        # Shape: (n_monos × 20) - solo los retornos que necesitas, sin ceros
        # NOTA: Esto es mucho más eficiente que construir matriz completa de pesos
        # (n_monos × n_assets) con mayoría de ceros y luego multiplicar matrices
        retornos_seleccionados = retornos_array[mes_idx, indices_seleccionados]
        
        # ========== OPTIMIZACIÓN 3: Calcular promedio equiponderado (equivalente a multiplicación de matrices) ==========
        # np.mean sobre axis=1 es matemáticamente equivalente a: sum(retornos × 0.05) para equiponderado
        # Porque: mean = sum/20 = sum × 0.05 (equivalente a pesos de 5% cada uno)
        # Esto evita construir matriz (n_monos × n_assets) con 42M elementos (solo 2.4% útiles)
        # y multiplicar (1 × 845) @ (845 × 50,000) cuando solo necesitamos trabajar con 20 activos
        retorno_mes_monos = np.mean(retornos_seleccionados, axis=1)
        # Shape: (n_monos,) - retorno promedio de los 20 activos para cada mono
        
        # ========== Aplicar coste de rebalanceo vectorizado ==========
        retorno_neto = retorno_mes_monos - COSTE_REBALANCEO
        
        # ========== Acumular retornos vectorizado ==========
        retornos_acumulados += retorno_neto
    
    # ========== CALCULAR CAGR VECTORIZADO ==========
    # Convertir a CAGR anualizado
    cagr_batch = (np.exp(retornos_acumulados) ** factor_cagr) - 1
    
    # Guardar resultados
    start_idx = batch_idx * BATCH_SIZE
    end_idx = min(start_idx + BATCH_SIZE, N_SIMULACIONES)
    resultados_cagr[start_idx:end_idx] = cagr_batch[:end_idx-start_idx]
    
    # Progress report
    simulaciones_procesadas = (batch_idx + 1) * BATCH_SIZE
    if simulaciones_procesadas % PROGRESS_INTERVAL == 0 or (batch_idx + 1) == n_batches:
        pct_completo = (simulaciones_procesadas / N_SIMULACIONES) * 100
        elapsed = time.time() - start_time
        rate = simulaciones_procesadas / elapsed if elapsed > 0 else 0
        print(f"Progreso: {pct_completo:>5.2f}% ({simulaciones_procesadas:,} / {N_SIMULACIONES:,} simulaciones) | "
              f"Velocidad: {rate:,.0f} sim/s | Tiempo: {elapsed:.1f}s")

elapsed_total = time.time() - start_time
print(f"\n{'='*70}")
print("MONTE CARLO COMPLETADO")
print(f"{'='*70}")
print(f"Total simulaciones: {N_SIMULACIONES:,}")
print(f"Batches procesados: {n_batches}")
print(f"Tiempo total: {elapsed_total:.2f} segundos ({elapsed_total/60:.2f} minutos)")
print(f"Velocidad promedio: {N_SIMULACIONES/elapsed_total:,.0f} simulaciones/segundo")


INICIANDO MONTE CARLO CON VECTORIZACIÓN COMPLETA OPTIMIZADA
Simulaciones: 25,000,000
Batch size: 50,000

Iniciando procesamiento de 500 batches (batch_size=50,000)...
Progress cada 0.1% (25,000 simulaciones)

OPTIMIZACIONES APLICADAS:
  - Extracción directa de retornos (solo n_monos × 20 elementos)
  - np.mean() en lugar de multiplicación de matrices (equivalente matemático)
  - Sin construcción de matriz completa de pesos (evita 42M elementos con ceros)
  - Selecciones generadas mes a mes (sin pre-cálculo innecesario)

Progreso:  0.20% (50,000 / 25,000,000 simulaciones) | Velocidad: 317 sim/s | Tiempo: 157.8s
Progreso:  0.40% (100,000 / 25,000,000 simulaciones) | Velocidad: 311 sim/s | Tiempo: 321.8s
Progreso:  0.60% (150,000 / 25,000,000 simulaciones) | Velocidad: 315 sim/s | Tiempo: 475.9s
Progreso:  0.80% (200,000 / 25,000,000 simulaciones) | Velocidad: 307 sim/s | Tiempo: 650.8s


KeyboardInterrupt: 

## 5. Análisis de Resultados {#resultados}

Análisis de los resultados de la simulación de Monte Carlo y comparación con la estrategia.

In [ ]:
# Calcular métricas de la estrategia (para comparación)
def calcular_metricas(equity_curve, retornos_mensuales, risk_free_rate=0.02):
    """Calcula métricas financieras básicas."""
    retornos = retornos_mensuales.dropna()
    años = (equity_curve.index[-1] - equity_curve.index[0]).days / 365.25
    total_return = (equity_curve.iloc[-1] / equity_curve.iloc[0]) - 1
    cagr = ((1 + total_return) ** (1 / años) - 1) * 100 if años > 0 else 0
    return cagr

# Calcular CAGR de la estrategia
cagr_estrategia = calcular_metricas(df_equity['equity'], df_equity['retorno_mensual']) / 100  # Convertir de % a decimal

# Comparar con estrategia
percentil = (resultados_cagr < cagr_estrategia).sum() / N_SIMULACIONES * 100

# Estadísticas
media_monos = resultados_cagr.mean()
mediana_monos = np.median(resultados_cagr)
std_monos = resultados_cagr.std()
p95 = np.percentile(resultados_cagr, 95)
p99 = np.percentile(resultados_cagr, 99)

print(f"\n{'='*70}")
print("RESULTADOS MONTE CARLO")
print(f"{'='*70}")
print(f"Media monos:      {media_monos*100:.2f}%")
print(f"Mediana monos:    {mediana_monos*100:.2f}%")
print(f"Std monos:        {std_monos*100:.2f}%")
print(f"Percentil 95:     {p95*100:.2f}%")
print(f"Percentil 99:     {p99*100:.2f}%")
print(f"\nEstrategia:       {cagr_estrategia*100:.2f}%")
print(f"Percentil:        {percentil:.2f}%")
print(f"\n¿Supera monos?    {'SÍ ✓' if percentil > 50 else 'NO ✗'}")

# Validaciones
assert resultados_cagr.min() > -0.9, "CAGR mínimo fuera de rango"
assert resultados_cagr.max() < 3.0, "CAGR máximo fuera de rango"
assert not np.isnan(resultados_cagr).any(), "NaN detectados"
assert not np.isinf(resultados_cagr).any(), "Inf detectados"

print("\n✓ Todas las validaciones pasadas")

In [ ]:
# Visualización
fig, ax = plt.subplots(figsize=(14, 6))
ax.hist(resultados_cagr * 100, bins=150, alpha=0.7, edgecolor='black', color='steelblue')
ax.axvline(cagr_estrategia * 100, color='red', linewidth=3,
           label=f'Estrategia: {cagr_estrategia*100:.2f}% (P{percentil:.1f})')
ax.axvline(media_monos * 100, color='orange', linewidth=2, linestyle='--',
           label=f'Media Monos: {media_monos*100:.2f}%')
ax.set_xlabel('CAGR (%)')
ax.set_ylabel('Frecuencia')
ax.set_title(f'Distribución de {N_SIMULACIONES:,} Portfolios Aleatorios (Vectorización Completa)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Comparación de Velocidad y Análisis Teórico {#comparacion}

### Comparación con Notebook 6

**Notebook 6 (Optimizado):**
- Vectorización de selección aleatoria con argsort
- Batch size: 50,000
- Loop sobre meses dentro de cada batch
- Cálculo de retornos usando indexing avanzado sobre array 3D

**Notebook 7 (Vectorización Completa Optimizada):**
- Extracción directa de retornos: `retornos_array[mes_idx, indices_seleccionados]` → `(n_monos × 20)`
- Cálculo de promedio equiponderado con `np.mean()` (equivalente matemático a multiplicación de matrices)
- Sin construcción de matriz completa de pesos (evita 42M elementos con mayoría de ceros)
- Selecciones generadas mes a mes (sin pre-cálculo innecesario)

### Optimizaciones Aplicadas y Justificación:

1. **Eliminación de matriz completa de pesos**:
   - **Antes**: Construía `weights_mes = np.zeros((50,000 × 845))` = 42,250,000 elementos
   - **Ahora**: Extrae directamente `(50,000 × 20)` = 1,000,000 elementos
   - **Reducción**: 97.6% menos memoria (solo 2.4% útil vs 100% de ceros)

2. **Equivalencia matemática de `np.mean()`**:
   - **Equiponderado**: Cada activo tiene peso 5% = 0.05
   - **Multiplicación de matrices**: `sum(retornos × 0.05)` para cada mono
   - **np.mean()**: `sum(retornos) / 20 = sum(retornos) × 0.05` (mismo resultado)
   - **Ventaja**: Más eficiente, trabaja solo con datos necesarios

3. **Eliminación de pre-cálculo de selecciones**:
   - **Antes**: Pre-calculaba todas las selecciones para todos los meses antes de calcular retornos
   - **Ahora**: Genera selecciones mes a mes cuando se necesitan
   - **Ventaja**: Reduce memoria y tiempo innecesario

4. **Extracción directa vs Multiplicación de matrices**:
   - **Antes**: `(1 × 845) @ (845 × 50,000)` con mayoría de ceros
   - **Ahora**: Extracción directa `(50,000 × 20)` y `np.mean(axis=1)`
   - **Ventaja**: Trabaja solo con datos relevantes, más rápido

### Ventajas de la Optimización:

1. **Memoria reducida**: De ~42M elementos a ~1M elementos por mes (97.6% reducción)
2. **Velocidad mejorada**: Elimina construcción de matrices grandes y multiplicaciones ineficientes
3. **Eficiencia**: Trabaja solo con los datos necesarios (20 activos vs 845 activos)
4. **Mismo resultado**: Matemáticamente equivalente, solo más eficiente

### Limitaciones:

- **Loop sobre meses**: Necesario por eligibility_mask variable mes a mes
- **Trade-off**: Optimización de memoria y velocidad manteniendo restricciones del problema

In [ ]:
# Análisis de memoria y eficiencia
print("\n" + "="*70)
print("ANÁLISIS DE EFICIENCIA")
print("="*70)

# Estimar memoria usada
BYTES_PER_FLOAT32 = 4
BYTES_PER_INT32 = 4
GB = 1024**3

# Memoria por batch (OPTIMIZADA - sin matriz completa de pesos)
mem_retornos_seleccionados = (BATCH_SIZE * N_ACTIVOS_SELECCION * BYTES_PER_FLOAT32) / GB  # Solo 20 activos
mem_selecciones = (BATCH_SIZE * N_ACTIVOS_SELECCION * BYTES_PER_INT32) / GB  # int32 para índices
mem_retornos_acumulados = (BATCH_SIZE * BYTES_PER_FLOAT32) / GB
mem_total_batch = mem_retornos_seleccionados + mem_selecciones + mem_retornos_acumulados

# Comparación con enfoque anterior (matriz completa de pesos)
mem_weights_anterior = (BATCH_SIZE * n_tickers * BYTES_PER_FLOAT32) / GB  # 50,000 × 845
reduccion_memoria = (1 - mem_total_batch / mem_weights_anterior) * 100

print(f"\nMemoria por batch (OPTIMIZADA):")
print(f"  Retornos seleccionados (n_monos × 20): {mem_retornos_seleccionados*1000:.2f} MB")
print(f"  Selecciones (índices): {mem_selecciones*1000:.2f} MB")
print(f"  Retornos acumulados: {mem_retornos_acumulados*1000:.2f} MB")
print(f"  Total: {mem_total_batch*1000:.2f} MB")

print(f"\nComparación con enfoque anterior:")
print(f"  Matriz completa de pesos (n_monos × n_assets): {mem_weights_anterior*1000:.2f} MB")
print(f"  Reducción de memoria: {reduccion_memoria:.1f}%")
print(f"  Eficiencia: Solo trabajamos con {N_ACTIVOS_SELECCION}/{n_tickers} = {(N_ACTIVOS_SELECCION/n_tickers*100):.1f}% de los activos")

print(f"\nMemoria total (resultados):")
mem_resultados = (N_SIMULACIONES * BYTES_PER_FLOAT32) / GB
print(f"  Resultados CAGR: {mem_resultados*1000:.2f} MB")

print(f"\nVelocidad de procesamiento:")
print(f"  Tiempo total: {elapsed_total:.2f} segundos ({elapsed_total/60:.2f} minutos)")
print(f"  Velocidad: {N_SIMULACIONES/elapsed_total:,.0f} simulaciones/segundo")
print(f"  Tiempo por batch: {elapsed_total/n_batches:.2f} segundos")

print(f"\n{'='*70}")
print("NOTEBOOK 7 COMPLETADO (OPTIMIZADO)")
print("="*70)
print("\nEste notebook demuestra cómo optimizaciones críticas pueden mejorar")
print("significativamente la velocidad y eficiencia de memoria de la simulación")
print("de Monte Carlo, manteniendo las mismas restricciones (eligibility_mask,")
print("rebalanceo mensual) y resultados matemáticamente equivalentes al Notebook 6.")
print("\nOPTIMIZACIONES CLAVE:")
print("  ✓ Extracción directa de retornos (solo n_monos × 20 elementos)")
print("  ✓ np.mean() en lugar de multiplicación de matrices (equivalente matemático)")
print("  ✓ Sin construcción de matriz completa de pesos (97.6% reducción de memoria)")
print("  ✓ Selecciones generadas mes a mes (sin pre-cálculo innecesario)")